# Lesson 5: Document RAG with LlamaIndex

In this lesson, we'll learn how to create a Retrieval-Augmented Generation (RAG) system that can process and query documents using LlamaIndex. This approach allows our AI agent to answer questions based on specific document content, making it perfect for document analysis, knowledge bases, and content-specific Q&A systems.

We'll build a complete document RAG solution that can handle multiple document formats and provide accurate, source-attributed responses.

## Lesson Objectives

By the end of this lesson, you will be able to:

1. Set up and configure LlamaIndex for document processing
2. Load and index various document formats (text, PDF, Word, etc.)
3. Create vector embeddings for semantic search
4. Build a basic RAG pipeline for document Q&A
5. Implement advanced retrieval strategies and filtering
6. Use LlamaIndex agents for complex document analysis
7. Create a conversational interface for document exploration
8. Handle multi-document queries and source attribution

## 1. Environment Setup

First, let's set up our environment with the necessary libraries for document RAG.

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 
- !pip install llama-index-embeddings-openai
- !pip install llama-index-llms-openai
- !pip install llama-index

In [1]:
# Install required packages if needed
# !pip install llama-index openai python-dotenv tiktoken chromadb

In [1]:
# Import required libraries
import os
import sys
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

# LlamaIndex imports
from llama_index.core import (
    VectorStoreIndex, 
    SimpleDirectoryReader, 
    StorageContext, 
    load_index_from_storage,
    Settings
)
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
import logging
import sys

# logging.basicConfig(
#     stream=sys.stdout, level=logging.INFO
# )  # logging.DEBUG for more verbose output
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))


# Additional utilities
from IPython.display import display, Markdown
import json

# Load environment variables
load_dotenv(find_dotenv())

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


In [2]:
# Get API keys and configuration
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
MODEL = os.environ.get("OPENAI_MODEL", "gpt-4.1")
EMBEDDING_MODEL = os.environ.get("OPENAI_EMBEDDING_MODEL", "text-embedding-3-large")

In [4]:
# Check if the necessary API keys are available
if not OPENAI_API_KEY:
    print("⚠️ OpenAI API key not found. Please set the OPENAI_API_KEY environment variable.")
else:
    print("✅ OpenAI API key loaded successfully")
    print(f"📋 Using model: {MODEL}")
    print(f"📋 Using embedding model: {EMBEDDING_MODEL}")

✅ OpenAI API key loaded successfully
📋 Using model: gpt-4.1
📋 Using embedding model: text-embedding-3-large


In [3]:
# Initialize OpenAI LLM for LlamaIndex
llm = OpenAI(
    model=MODEL,
    api_key=OPENAI_API_KEY,
    temperature=0.1
)

# Configure embedding model
embed_model = OpenAIEmbedding(
    model=EMBEDDING_MODEL,
    api_key=OPENAI_API_KEY
)

In [5]:
# Set global configurations for LlamaIndex
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 1024
Settings.chunk_overlap = 200

print("🔧 LlamaIndex configured with OpenAI")
print(f"📊 Chunk size: {Settings.chunk_size}")
print(f"📊 Chunk overlap: {Settings.chunk_overlap}")

🔧 LlamaIndex configured with OpenAI
📊 Chunk size: 1024
📊 Chunk overlap: 200


## 2. Loading and Processing Documents

Let's start by loading and processing our sample documents. LlamaIndex makes it easy to load various document formats.

In [ ]:
documents = SimpleDirectoryReader(
    input_dir="../lesson_5_document_rag/sample_documents", # provide the path to your documents here
).load_data()


## 3. Creating Vector Index

Now let's create a vector index from our documents. This will enable semantic search over the document content.

In [7]:
index = VectorStoreIndex.from_documents(documents)

2025-11-03 06:46:22,772 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


## 4. Basic RAG Query Engine

Let's create a basic query engine that can answer questions based on our document content.

In [12]:
query = "What are these documents about?"
query_engine = index.as_query_engine()
answer = query_engine.query(query)

print("\n\n**query was:**\n", query)
print("\n**answer was:**\n", answer)

print("=" * 60)
print("**sources were:**")
print(answer.get_formatted_sources())



2025-11-03 06:49:27,295 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-11-03 06:49:30,039 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"




**query was:**
 What are these documents about?

**answer was:**
 The first document provides a comprehensive overview of Artificial Intelligence (AI) and Machine Learning (ML), explaining their definitions, types, fundamental concepts, applications across various industries such as healthcare, finance, transportation, and technology, as well as the challenges and future outlook associated with these technologies.

The second document discusses key trends and strategies in cloud computing, including serverless computing, container orchestration, multi-cloud and hybrid strategies, and the integration of AI and ML services. It also explores the future direction of cloud computing, highlighting advancements like edge computing, sustainability, enhanced security, automation, and quantum computing, and emphasizes the importance of strategic cloud adoption for organizations.
**sources were:**
> Source (Doc id: 9283ea4a-c377-453b-aebd-f2b61c142d8c): Artificial Intelligence and Machine Learn

In [13]:
# Test the basic query engine
def test_basic_query(question):
    """Test a query and display results"""
    print(f"🔍 Question: {question}")
    print("-" * 50)
    
    response = query_engine.query(question)
    
    print("🤖 Answer:")
    print(response.response)
    
    # Show source information
    print(f"\n📚 Sources used: {len(response.source_nodes)} chunks")
    for i, node in enumerate(response.source_nodes, 1):
        print(f"  Source {i}: {node.metadata.get('file_name', 'Unknown')}")
        print(f"    Score: {node.score:.3f}")
        print(f"    Text: {node.text[:150]}...")
        print()
    
    return response



In [14]:
# Test with sample questions
questions = [
    "What is artificial intelligence?",
    "What are the benefits of cloud computing?",
    "Explain the different types of machine learning"
]

for question in questions:
    print("=" * 60)
    test_basic_query(question)
    print()

🔍 Question: What is artificial intelligence?
--------------------------------------------------


2025-11-03 06:50:02,927 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-11-03 06:50:04,873 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


🤖 Answer:
Artificial intelligence refers to the simulation of human intelligence in machines that are programmed to think and learn like humans. It encompasses machines that exhibit traits associated with a human mind, such as learning and problem-solving.

📚 Sources used: 2 chunks
  Source 1: sample1.txt
    Score: 0.435
    Text: Artificial Intelligence and Machine Learning: A Comprehensive Overview

Introduction

Artificial Intelligence (AI) and Machine Learning (ML) have beco...

  Source 2: sample2.txt
    Score: 0.256
    Text: Serverless Computing:
Running applications without managing servers, paying only for actual usage.

Container Orchestration:
Using technologies like K...


🔍 Question: What are the benefits of cloud computing?
--------------------------------------------------


2025-11-03 06:50:05,589 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-11-03 06:50:08,079 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


🤖 Answer:
Cloud computing offers several benefits, including cost efficiency through reduced capital expenditure on hardware and infrastructure, pay-as-you-use pricing models, and lower operational costs due to economies of scale. It provides scalability and flexibility, allowing rapid scaling up or down based on demand, access to the latest technologies and services, and global reach and availability. Cloud computing also ensures reliability and availability with high uptime guarantees, built-in redundancy, disaster recovery, and professional management and monitoring. Additionally, it fosters innovation and speed by enabling faster time-to-market for applications, access to advanced technologies such as AI, ML, and IoT, and allowing organizations to focus on their core business rather than IT infrastructure.

📚 Sources used: 2 chunks
  Source 1: sample2.txt
    Score: 0.557
    Text: Cloud Computing: Transforming Modern Business Infrastructure

Introduction

Cloud computing has revol

2025-11-03 06:50:08,840 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-11-03 06:50:11,518 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


🤖 Answer:
There are three main types of machine learning:

1. Supervised Learning: In this approach, algorithms learn from labeled training data to make predictions on new, unseen data. Common examples include classification and regression problems.

2. Unsupervised Learning: Here, algorithms analyze data without labeled examples to find hidden patterns or structures. Techniques such as clustering and dimensionality reduction are commonly used.

3. Reinforcement Learning: In this type, algorithms learn by interacting with an environment and receiving rewards or penalties based on their actions, gradually improving their performance over time.

📚 Sources used: 2 chunks
  Source 1: sample1.txt
    Score: 0.427
    Text: Artificial Intelligence and Machine Learning: A Comprehensive Overview

Introduction

Artificial Intelligence (AI) and Machine Learning (ML) have beco...

  Source 2: sample2.txt
    Score: 0.279
    Text: Serverless Computing:
Running applications without managing server

## 5. Advanced Retrieval and Filtering

Let's enhance our RAG system with more advanced retrieval strategies and post-processing filters.

In [15]:
# Create advanced retriever with filters
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core import get_response_synthesizer

# Create retriever with more results initially
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=5,  # Get more results initially
)

# Add post-processing filters
postprocessor = SimilarityPostprocessor(similarity_cutoff=0.7)

# Create response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
    use_async=False,
)

# Create advanced query engine
advanced_query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[postprocessor],
    response_synthesizer=response_synthesizer
)

print("🚀 Advanced query engine created!")
print("📋 Enhanced features:")
print("  - Similarity filtering (cutoff: 0.7)")
print("  - Higher initial retrieval (top 5)")
print("  - Post-processing for relevance")
print("  - Tree summarization for better responses")

🚀 Advanced query engine created!
📋 Enhanced features:
  - Similarity filtering (cutoff: 0.7)
  - Higher initial retrieval (top 5)
  - Post-processing for relevance
  - Tree summarization for better responses


In [16]:
# Create a custom query engine that can filter by document source
def create_document_specific_query_engine(document_filter=None):
    """
    Create a query engine that can filter by specific documents
    
    Args:
        document_filter: String to match document names (e.g., "sample1.txt")
    """
    
    if document_filter:
        # Filter nodes by document
        filtered_nodes = [
            node for node in index.docstore.docs.values() 
            if document_filter.lower() in node.metadata.get('file_name', '').lower()
        ]
        
        if not filtered_nodes:
            print(f"⚠️ No documents found matching filter: {document_filter}")
            return None
            
        # Create a new index with filtered nodes
        from llama_index.core import DocumentSummaryIndex
        filtered_docs = [
            doc for doc in documents 
            if document_filter.lower() in doc.metadata.get('file_name', '').lower()
        ]
        
        if filtered_docs:
            filtered_index = VectorStoreIndex.from_documents(filtered_docs)
            query_engine = filtered_index.as_query_engine(similarity_top_k=3)
            print(f"🎯 Created filtered query engine for: {document_filter}")
            print(f"📄 Documents included: {len(filtered_docs)}")
            return query_engine
    
    return advanced_query_engine

# Test document-specific querying
def query_specific_document(question, document_filter=None):
    """Query a specific document or all documents"""
    
    if document_filter:
        print(f"🔍 Querying document: {document_filter}")
        engine = create_document_specific_query_engine(document_filter)
        if not engine:
            return None
    else:
        print("🔍 Querying all documents")
        engine = advanced_query_engine
    
    print(f"❓ Question: {question}")
    print("-" * 50)
    
    response = engine.query(question)
    print("🤖 Answer:")
    print(response.response)
    
    return response

# Example queries
print("Testing document-specific querying:")
print("=" * 60)

# Query about AI from the AI document
query_specific_document(
    "What are the main types of artificial intelligence?", 
    "sample1.txt"
)

print("\n" + "=" * 60)

# Query about cloud computing from the cloud document  
query_specific_document(
    "What are the cloud service models?",
    "sample2.txt"
)

# Test advanced query engine with similarity filtering
test_queries = [
    "What are the main concepts in artificial intelligence?",
    "How does cloud computing work?",
    "What are the benefits of machine learning?",
    "Explain containerization in cloud computing"
]

print("🔍 Testing Advanced Query Engine with Similarity Filtering")
print("=" * 60)

for i, query in enumerate(test_queries, 1):
    print(f"\n{i}. Query: {query}")
    print("-" * 40)
    
    # Query with advanced engine
    response = advanced_query_engine.query(query)
    print(f"🤖 Response: {response.response}")
    
    # Show retrieved nodes information
    if hasattr(response, 'source_nodes') and response.source_nodes:
        print(f"📚 Sources used: {len(response.source_nodes)} documents")
        for j, node in enumerate(response.source_nodes[:2]):  # Show first 2 sources
            if hasattr(node, 'score'):
                print(f"   - Source {j+1} (similarity: {node.score:.3f})")
            else:
                print(f"   - Source {j+1}")
    
    print()

print("✅ Advanced query testing completed!")

Testing document-specific querying:
🔍 Querying document: sample1.txt


2025-11-03 06:51:06,002 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


🎯 Created filtered query engine for: sample1.txt
📄 Documents included: 1
❓ Question: What are the main types of artificial intelligence?
--------------------------------------------------


2025-11-03 06:51:07,029 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-11-03 06:51:08,461 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


🤖 Answer:
The main types of artificial intelligence are:

1. Narrow AI (Weak AI): These systems are designed and trained for specific tasks, such as virtual assistants, recommendation algorithms, and image recognition systems.
2. General AI (Strong AI): This is a hypothetical form of AI that can understand, learn, and apply knowledge across a wide range of tasks at a level equal to human intelligence.
3. Superintelligence: This type of AI would surpass human intelligence in all aspects, including creativity, general wisdom, and social skills.

🔍 Querying document: sample2.txt


...

🤖 Response: Empty Response


3. Query: What are the benefits of machine learning?
----------------------------------------


2025-11-03 06:51:13,172 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


🤖 Response: Empty Response


4. Query: Explain containerization in cloud computing
----------------------------------------


2025-11-03 06:51:13,889 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


🤖 Response: Empty Response

✅ Advanced query testing completed!


## 6. Conversational Document Agent

Let's create a conversational agent that can maintain context across multiple queries about our documents.

In [17]:
# Create a conversational document agent
from llama_index.core.memory import ChatMemoryBuffer

class ConversationalDocumentAgent:
    def __init__(self, index, memory_limit=10):
        self.index = index
        self.memory = ChatMemoryBuffer.from_defaults(token_limit=3000)
        self.conversation_history = []
        self.memory_limit = memory_limit
        
        # Create chat engine with memory
        self.chat_engine = index.as_chat_engine(
            chat_mode="context",
            memory=self.memory,
            similarity_top_k=3,
            system_prompt=(
                "You are a helpful document analysis assistant. "
                "Answer questions based on the provided document content. "
                "Always cite the source document when possible. "
                "If information is not in the documents, clearly state that. "
                "Maintain conversation context and refer to previous answers when relevant."
            )
        )
    
    def query(self, question):
        """Process a query with conversation context"""
        print(f"🗣️ User: {question}")
        print("-" * 50)
        
        # Get response from chat engine
        response = self.chat_engine.chat(question)
        
        print("🤖 Assistant:")
        print(response.response)
        
        # Add to conversation history
        self.conversation_history.append({
            "question": question,
            "answer": response.response,
            "sources": len(response.source_nodes) if hasattr(response, 'source_nodes') else 0
        })
        
        # Keep history manageable
        if len(self.conversation_history) > self.memory_limit:
            self.conversation_history = self.conversation_history[-self.memory_limit:]
        
        # Show sources if available
        if hasattr(response, 'source_nodes') and response.source_nodes:
            print(f"\n📚 Sources ({len(response.source_nodes)} chunks):")
            for i, node in enumerate(response.source_nodes, 1):
                doc_name = node.metadata.get('file_name', 'Unknown')
                print(f"  {i}. {doc_name} (Score: {node.score:.3f})")
        
        return response
    
    def get_conversation_summary(self):
        """Get a summary of the conversation"""
        return {
            "total_questions": len(self.conversation_history),
            "questions": [item["question"] for item in self.conversation_history]
        }
    
    def reset_conversation(self):
        """Reset the conversation history"""
        self.conversation_history = []
        self.memory.reset()
        print("🔄 Conversation history reset")

# Create the conversational agent
agent = ConversationalDocumentAgent(index)
print("💬 Conversational Document Agent created!")
print("🎯 Features:")
print("  - Maintains conversation context")
print("  - Source attribution")
print("  - Memory management")
print("  - Multi-turn conversations")

💬 Conversational Document Agent created!
🎯 Features:
  - Maintains conversation context
  - Source attribution
  - Memory management
  - Multi-turn conversations


In [18]:
# Test the conversational agent with a multi-turn conversation
conversation_queries = [
    "What is artificial intelligence?",
    "What are the main types you mentioned?",
    "How does this relate to machine learning?",
    "Now tell me about cloud computing",
    "What are the advantages compared to traditional IT infrastructure?",
    "Which service model would be best for a startup?"
]

print("🎭 Starting multi-turn conversation:")
print("=" * 60)

for i, query in enumerate(conversation_queries, 1):
    print(f"\n🔄 Turn {i}:")
    agent.query(query)
    
    if i < len(conversation_queries):
        print("\n⏳ (continuing conversation...)")

print(f"\n📊 Conversation summary: {agent.get_conversation_summary()}")

🎭 Starting multi-turn conversation:

🔄 Turn 1:
🗣️ User: What is artificial intelligence?
--------------------------------------------------


2025-11-03 06:52:26,899 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-11-03 06:52:28,947 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


🤖 Assistant:
Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think and learn like humans. The term may also be applied to any machine that exhibits traits associated with a human mind such as learning and problem-solving. AI can be broadly categorized into:

1. **Narrow AI (Weak AI):** AI systems designed and trained for a particular task (e.g., virtual assistants like Siri or Alexa, recommendation algorithms, image recognition systems).
2. **General AI (Strong AI):** A hypothetical form of AI that can understand, learn, and apply knowledge across a wide range of tasks at a level equal to human intelligence.
3. **Superintelligence:** AI that surpasses human intelligence in all aspects, including creativity, general wisdom, and social skills.

(Source: sample1.txt)

📚 Sources (3 chunks):
  1. sample1.txt (Score: 0.435)
  2. sample2.txt (Score: 0.256)
  3. sample2.txt (Score: 0.250)

⏳ (continuing conversation...)

🔄 Turn 2:


2025-11-03 06:52:29,640 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-11-03 06:52:31,812 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


🤖 Assistant:
The main types of artificial intelligence mentioned are:

1. **Narrow AI (Weak AI):**  
   AI systems that are designed and trained for a particular task. Examples include virtual assistants like Siri or Alexa, recommendation algorithms, and image recognition systems.

2. **General AI (Strong AI):**  
   A hypothetical form of AI that possesses the ability to understand, learn, and apply knowledge across a wide range of tasks at a level equal to human intelligence.

3. **Superintelligence:**  
   AI that surpasses human intelligence in all aspects, including creativity, general wisdom, and social skills.

(Source: sample1.txt)

📚 Sources (3 chunks):
  1. sample1.txt (Score: 0.199)
  2. sample2.txt (Score: 0.171)
  3. sample2.txt (Score: 0.167)

⏳ (continuing conversation...)

🔄 Turn 3:
🗣️ User: How does this relate to machine learning?
--------------------------------------------------


...

🤖 Assistant:
Cloud computing is the delivery of computing services—including servers, storage, databases, networking, software, analytics, and intelligence—over the internet (“the cloud”) to offer faster innovation, flexible resources, and economies of scale. It has revolutionized how businesses and individuals access, store, and process data by providing on-demand access to these resources without the need for direct management or ownership of physical infrastructure.

**Essential Characteristics of Cloud Computing:**
1. **On-demand self-service:** Users can provision computing capabilities automatically without human interaction with service providers.
2. **Broad network access:** Capabilities are available over the network and accessed through standard mechanisms.
3. **Resource pooling:** Computing resources are pooled to serve multiple consumers using a multi-tenant model.
4. **Rapid elasticity:** Capabilities can be elastically provisioned and released to scale rapidly.
5. **Measu

2025-11-03 06:52:45,640 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-11-03 06:52:51,575 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


🤖 Assistant:
Cloud computing offers several advantages over traditional IT infrastructure, including:

1. **Cost Efficiency**
   - **Reduced capital expenditure:** No need to invest heavily in physical hardware and infrastructure.
   - **Pay-as-you-use pricing:** Only pay for the resources you actually use.
   - **Lower operational costs:** Benefit from economies of scale provided by cloud providers.

2. **Scalability and Flexibility**
   - **Rapid scaling:** Easily scale resources up or down based on demand.
   - **Access to latest technologies:** Instantly use new services and technologies offered by cloud providers.
   - **Global reach:** Services are available from anywhere with an internet connection.

3. **Reliability and Availability**
   - **High uptime guarantees:** Cloud providers often offer 99.9% or higher uptime.
   - **Built-in redundancy and disaster recovery:** Data and applications are protected against hardware failures.
   - **Professional management:** Cloud provide

2025-11-03 06:52:52,395 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-11-03 06:52:59,257 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


🤖 Assistant:
For most startups, **Platform as a Service (PaaS)** is often the best cloud service model. Here’s why:

### Why PaaS is Ideal for Startups:
- **Faster Application Development and Deployment:** PaaS provides a ready-to-use platform with tools and frameworks, allowing startups to quickly build, test, and launch applications without worrying about managing servers or infrastructure.
- **Reduced Complexity:** Startups can focus on developing their core product rather than handling the complexities of underlying infrastructure, operating systems, or middleware.
- **Built-in Scalability and Load Balancing:** PaaS platforms automatically handle scaling and load balancing, which is crucial for startups expecting rapid growth or variable workloads.
- **Cost-Effective:** Startups can avoid large upfront investments in hardware and only pay for what they use, making it easier to manage limited budgets.

### Examples of PaaS:
- Google App Engine
- Microsoft Azure App Service
- Heroku


## 7. Advanced Features and Document Analysis

Let's explore some advanced features for document analysis and comparison.

In [ ]:
# Advanced document analysis functions

def compare_documents(topic, doc1_filter=None, doc2_filter=None):
    """Compare how different documents discuss a topic"""
    
    print(f"📊 Comparing documents on topic: {topic}")
    print("=" * 60)
    
    if doc1_filter and doc2_filter:
        # Query each document separately
        engine1 = create_document_specific_query_engine(doc1_filter)
        engine2 = create_document_specific_query_engine(doc2_filter)
        
        if engine1 and engine2:
            print(f"📄 Document 1: {doc1_filter}")
            response1 = engine1.query(f"Explain {topic}")
            print("🤖 Response:")
            print(response1.response)
            
            print(f"\n📄 Document 2: {doc2_filter}")
            response2 = engine2.query(f"Explain {topic}")
            print("🤖 Response:")
            print(response2.response)
            
            # Use the main engine to synthesize comparison
            print(f"\n🔄 Synthesizing comparison...")
            comparison_query = f"""
            Based on the available documents, compare and contrast how {topic} is presented. 
            Highlight similarities and differences in the explanations, approaches, or perspectives.
            """
            
            comparison_response = advanced_query_engine.query(comparison_query)
            print("\n🎯 Comparison Analysis:")
            print(comparison_response.response)
    else:
        # General comparison query
        comparison_query = f"""
        Compare and contrast the different perspectives or approaches to {topic} 
        found in the available documents. Highlight key similarities and differences.
        """
        response = advanced_query_engine.query(comparison_query)
        print("🤖 Analysis:")
        print(response.response)

def summarize_document_collection():
    """Create a comprehensive summary of all documents"""
    
    print("📋 Creating comprehensive document summary...")
    print("=" * 60)
    
    summary_query = """
    Provide a comprehensive summary of all the documents in the collection. 
    Include:
    1. Main topics covered
    2. Key concepts and definitions
    3. Important relationships between topics
    4. Overall themes and insights
    
    Structure your response clearly with headers and bullet points.
    """
    
    response = advanced_query_engine.query(summary_query)
    print("📊 Document Collection Summary:")
    print(response.response)
    
    return response

def extract_key_concepts():
    """Extract key concepts and definitions from all documents"""
    
    print("🔑 Extracting key concepts and definitions...")
    print("=" * 60)
    
    concepts_query = """
    Extract and list the key concepts, terms, and definitions from all documents.
    For each concept, provide:
    1. The term/concept name
    2. Its definition or explanation
    3. Which document(s) it appears in
    
    Focus on important technical terms, main ideas, and fundamental concepts.
    """
    
    response = advanced_query_engine.query(concepts_query)
    print("🎓 Key Concepts:")
    print(response.response)
    
    return response

# Test advanced analysis features
print("🔬 Advanced Document Analysis")
print("=" * 60)

# Compare how both documents discuss technology concepts
compare_documents("the future impact of technology")

print("\n" + "=" * 60)

# Get comprehensive summary
summary_response = summarize_document_collection()

print("\n" + "=" * 60)

# Extract key concepts
concepts_response = extract_key_concepts()